# Introduction

This is a follow-up on the data preperation notebook. In this notebook you can find our experiments with the model and optimalisations.

The notebook is setup as followed:

1. Imports (importing necessary libraries to run the code)
1. Functions (creating necessary functions to run the code)
1. Final data preperations (final data preperation steps taken before training the model)
1. Model 1 (first version of the Artificial Neural Network (ANN) model used to create the SDM)
1. Model 2 (second version of the ANN modek used to create the SDM)
1. Model 3 (final base model)
1. Visualize the predictions with the final base model
1. Visualisations with SDM optimalisations

# 1. Imports
Importing necessairy libraries to run the code

In [ ]:
import numpy as np 
import pandas as pd
import geopandas as gpd

import tensorflow as tf
print(f'Tensorflow version: {tf.__version__}')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from imblearn.over_sampling import RandomOverSampler

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
repo_path = Path.cwd().parent

# 2. Functions
Creating necessary functions to run the code. Not all functions are mentioned here for practical reasons.

In [ ]:
# Create categorical values (from string to int)
def create_cat_values(column, column_name):
    """
    column      = the column of a dataframe that will be created categorical values of
    column_name = the name of the column
    """
    column = column.copy()
    column = column.astype('category')
    d = column.cat.categories
    column = column.cat.codes
    uniques = len(column.unique())
    print(f'{column_name} contains {str(uniques)} categories')
    return column, d

# Plotting the loss of a model
def plot_loss(model_name, model):
    loss = model.history['loss']
    plt.plot(loss)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.title(f'Loss of model {model_name} = {min(loss)}')
    plt.show()

# Plotting the accuracy of a model
def plot_accuracy(model_name, model):
    acc = model.history['accuracy']
    plt.plot(acc)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.title(f'Accuracy of model {model_name} = {max(acc)}')
    plt.show()
    
# Plotting the sparce top-k categorical accuracy of a model
def plot_sparse_accuracy(model_name, model):
    acc = model.history['sparse_top_k_categorical_accuracy']
    plt.plot(acc)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.title(f'Accuracy of model {model_name} = {max(acc)}')
    plt.show()
    
# Plotting the correlation matrix of a DataFrame
def plot_correlation_matrix(dataframe):
    corrmatrix = dataframe.corr()
    mask = np.zeros_like(corrmatrix)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):
        f, ax = plt.subplots(figsize=(7, 5))
        ax = sns.heatmap(corrmatrix, mask=mask, vmax=.3, square=True)

def get_index_of_observations(y_test):
    
    np.asarray(y_test).astype('float32').reshape((-1,5))
    
    i = 0

    i_list_vlinder_1 = []
    i_list_vlinder_2 = []
    i_list_vlinder_3 = []
    i_list_vlinder_4 = []
    i_list_vlinder_5 = []

    for items in y_test:
        v1 = items[0]
        v2 = items[1]
        v3 = items[2]
        v4 = items[3]
        v5 = items[4]

        if v1 == 1:
            i_list_vlinder_1.append(i)

        if v2 == 1:
            i_list_vlinder_2.append(i)

        if v3 == 1:
            i_list_vlinder_3.append(i)

        if v4 == 1:
            i_list_vlinder_4.append(i)

        if v5 == 1:
            i_list_vlinder_5.append(i)

        i = i + 1
    return i_list_vlinder_1, i_list_vlinder_2, i_list_vlinder_3, i_list_vlinder_4, i_list_vlinder_5


def threshold(threshold, y2_pred):
    
    y2_pred_threshold = y2_pred.copy()

    for idx1, a in enumerate(y2_pred):
        for idx2, element in enumerate(a):

            if a[idx2] > threshold:
                y2_pred_threshold[idx1, idx2] = 1

            else:
                y2_pred_threshold[idx1, idx2] = 0
    
    return y2_pred_threshold

def predict_accuracy(y_test, y2_pred_threshold, butterflies):
    
    acc_vlinder_1 = accuracy_score_butterfly1 = accuracy_score(y_test[butterflies[0], 0], y2_pred_threshold[butterflies[0], 0])
    acc_vlinder_2 = accuracy_score_butterfly2 = accuracy_score(y_test[butterflies[1], 1], y2_pred_threshold[butterflies[1], 1])
    acc_vlinder_3 = accuracy_score_butterfly3 = accuracy_score(y_test[butterflies[2], 2], y2_pred_threshold[butterflies[2], 2])
    acc_vlinder_4 = accuracy_score_butterfly4 = accuracy_score(y_test[butterflies[3], 3], y2_pred_threshold[butterflies[3], 3])
    acc_vlinder_5 = accuracy_score_butterfly5 = accuracy_score(y_test[butterflies[4], 4], y2_pred_threshold[butterflies[4], 4])
    
    return acc_vlinder_1, acc_vlinder_2, acc_vlinder_3, acc_vlinder_4, acc_vlinder_5


def scatter_plot(x, y, x_label, y_label, title, threshold):
    plt.scatter(x, y)
    plt.axhline(y=threshold, color='r', linestyle='-')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.show()
    
def boxplot(my_dict,x_label, y_label, title,threshold):
    fig, ax = plt.subplots()
    ax.boxplot(my_dict.values())
    ax.set_xticklabels(my_dict.keys())
    plt.axhline(y=threshold, color='r', linestyle='-')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

def analyse(X_train, X_test, y_train, y_test, model, thresh):
    y_train, y_test = pd.get_dummies(y_train), pd.get_dummies(y_test)
    y_test = np.asarray(y_test).astype('float32').reshape((-1,5))
    y_pred = model.predict(X_test)

    i_list_vlinder_1, i_list_vlinder_2, i_list_vlinder_3, i_list_vlinder_4, i_list_vlinder_5 = get_index_of_observations(y_test)
    
    butterflies = []
    butterflies.append(i_list_vlinder_1)
    butterflies.append(i_list_vlinder_2)
    butterflies.append(i_list_vlinder_3)
    butterflies.append(i_list_vlinder_4)
    butterflies.append(i_list_vlinder_5)

    y_pred_threshold = threshold(thresh, y_pred)

    accuracy_score_butterfly1, accuracy_score_butterfly2, accuracy_score_butterfly3, accuracy_score_butterfly4, accuracy_score_butterfly5 = predict_accuracy(y_test, y_pred_threshold, butterflies)

    y_pred_threshold = threshold(thresh, y_pred)

    
    #scatterplots
    scatter_plot(i_list_vlinder_1, y_pred[butterflies[0],0], "", "prediction value", "scatterplot butterfly 1 acc: " + str(accuracy_score_butterfly1), thresh)
    scatter_plot(i_list_vlinder_2, y_pred[butterflies[1],1], "", "prediction value", "scatterplot butterfly 2 acc: " + str(accuracy_score_butterfly2), thresh)
    scatter_plot(i_list_vlinder_3, y_pred[butterflies[2],2], "", "prediction value", "scatterplot butterfly 3 acc: " + str(accuracy_score_butterfly3), thresh)
    scatter_plot(i_list_vlinder_4, y_pred[butterflies[3],3], "", "prediction value", "scatterplot butterfly 4 acc: " + str(accuracy_score_butterfly4), thresh)
    scatter_plot(i_list_vlinder_5, y_pred[butterflies[4],4], "", "prediction value", "scatterplot butterfly 5 acc: " + str(accuracy_score_butterfly5), thresh)
    
    #boxplot
    my_dict = {'v1': y_pred[i_list_vlinder_1,0], 
           'v2': y_pred[i_list_vlinder_2,1], 
           'v3': y_pred[i_list_vlinder_3,2], 
           'v4': y_pred[i_list_vlinder_4,3], 
           'v5': y_pred[i_list_vlinder_5,4]}
    
    boxplot(my_dict,"", "prediction value", "boxplot of all butterflies", thresh)
    
    #classification report
    print(classification_report(y_test, y_pred_threshold))

# 3. Final data preperations
## From laurens we recieved the following points of feedback:
1. Make use of different loss and accuracy functions
1. Make a selection of two types of butterflies
1. Less layers to train with
1. Keep decreasing the number of neurons as the layers continue
1. Check the output
1. We dont have a classification problem but a multilabel-problem.

Accuracy:
https://keras.io/api/metrics/accuracy_metrics/#sparsetopkcategoricalaccuracy-class
<br>
Loss:
https://keras.io/api/losses/probabilistic_losses/#binarycrossentropy-class

In [ ]:
df = pd.read_csv(repo_path / 'data/prepared/data_merged.csv', index_col=0)

## Creating a selection of a few butterflies with different patterns (POC)

The selection was made by Vincent Kalkman, which is a known biologist from Naturalis and gave us a list of butterflies that we can use:
- Parnassius apollo 
- Zerynthia rumina
- Agriades optilete
- Melanargia galathea
- Boloria titania 

By using a subset with only 5 butterflies, it is much easier to make a proof of concept because now, it will take less time to train the model, and also these butterflies are all located at different parts of Europe, so that will make the improvements made to the model easier to spot.

Below we will:

1. Make a DataFrame with only the selected butterflies (only for POC).
1. Count the amount of each butterfly.
1. Change the butterfly name into categorical data, to make it easier to distinguish for machines (from words to numbers).

**Only run the following code when runing the proof of concept (POC)**

In [ ]:
###################################################################
#      To run on the full dataset, comment out the code below     #
###################################################################

butterfly_list = ['Parnassius apollo', 
                  'Zerynthia rumina', 
                  'Agriades optilete',
                  'Melanargia galathea', 
                  'Boloria titania']

# Only use selected butterflies
df = df[df['taxon_full_name'].isin(butterfly_list)]

####################################################################

## Continuation of final data preperation

In [ ]:
# Count unique values
df['taxon_full_name'].value_counts()

In [ ]:
# Transfrom species names and land climate names into categorial values
df['taxon_full_name'], categories_taxon = create_cat_values(df['taxon_full_name'], 'taxon_full_name')
df.head()

## Split the data into input (X) and output (y) values

In [ ]:
# Features X gets all columns except first column
X = df.iloc[:, 1:]

# Label y gets first column which is 'taxon_full_name'
y = df.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# 4. Model 1 with SpargeTopKCategoricalAccuracy (k=1)

This was the first version of our Artificial Neural Network (ANN) that was used to create the SDM output. This network used `SpargeTopKCategoricalAccuracy` and `sparse_categorical_crossentropy` as metrics with `k=1`.

**This was the first version and is not used any further.**

In [ ]:
# This is the function used to train the first network
def train_model1(X_train, X_test, y_train, y_test):
    output_neurons = len(y_train.unique())

    model = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_uniform'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(56, activation='relu'),
      tf.keras.layers.Dense(28, activation='relu'),
      tf.keras.layers.Dense(output_neurons, activation='sigmoid')
    ])

    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    acc = tf.keras.metrics.SparseTopKCategoricalAccuracy(
        k=1, name="sparse_top_k_categorical_accuracy", dtype=None
    )

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=[acc])  
                  
    model_history = model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200)

    plot_sparse_accuracy('Model 1 (sparse_categorical_crossentropy 1)', model_history)
    plot_loss('Model 1 (sparse_categorical_crossentropy 1)', model_history)

    return model1

# Uncomment the line below to run the function
# model1 = train_model1(X_train, X_test, y_train, y_test)

# 5. Model 2 with SpargeTopKCategoricalAccuracy (k=2)

This was the second version of our ANN that was used to create the SDM output. This network used `SpargeTopKCategoricalAccuracy` and `sparse_categorical_crossentropy` as metrics with `k=2`.

**This model is used to compare the perfomance between different metrics.**

In [ ]:
# This is the function used to train the second network
def train_model2(X_train, X_test, y_train, y_test):
    output_neurons = len(y_train.unique())

    model = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_uniform'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(56, activation='relu'),
      tf.keras.layers.Dense(28, activation='relu'),
      tf.keras.layers.Dense(output_neurons, activation='sigmoid')
    ])

    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    acc = tf.keras.metrics.SparseTopKCategoricalAccuracy(
        k=2, name="sparse_top_k_categorical_accuracy", dtype=None
    )

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=[acc])  

    model_history = model.fit(X_train, 
                              y_train, 
                              validation_data=(X_test, y_test), 
                              epochs=200)

    plot_sparse_accuracy('Model 2 (sparse_categorical_crossentropy 2)', model_history)
    plot_loss('Model 2 (sparse_categorical_crossentropy 2)', model_history)

    return model

model2 = train_model2(X_train, X_test, y_train, y_test)

# 6. Model 3 with Binary-Crossentropy (final base model, without optimalisations)
Below you can see how we trained the final base model. This model has 5 Dense layers, wich 256, 128, 56, and 28 neurons respectively. The first 4 use relu activation, and the last uses sigmoid activation. 

We are also using the optimizer Adam with a learning rate of 0.001, the loss function binary-crossentropy, and with the regular accuracy metric.

This function can also be enabled to use oversampling, normalizing the amount of each butterfly, to make them equal. However, this is not used right now but will be used later on in the notebook.

In [ ]:
# This is the function used to train the final base model
def train_model3(X_train, X_test, y_train, y_test, oversample=False, verbose=1):
  """
  X_train    = data that the model is trained with
  X_test     = data that the model is tested with
  y_train    = the species you are trying to predict during training
  y_test     = the species you are trying to predict during testing
  oversample = enables oversampling of the training data
  verbose    = 1: prints training info. 0: does not print output training info
  """
  y_train, y_test = pd.get_dummies(y_train), pd.get_dummies(y_test)
  output_neurons = len(y_train.columns)

  if oversample:
    oversampler = RandomOverSampler()
    X_train, y_train = oversampler.fit_resample(X_train.to_numpy(), y_train.to_numpy())
    output_neurons = len(y_train[0])

  model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, 
                      activation='relu',
                      kernel_initializer='he_uniform'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(56, activation='relu'),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dense(output_neurons, activation='sigmoid')
  ])

  opt = tf.keras.optimizers.Adam(learning_rate=0.001)

  model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
  
  model_history = model.fit(X_train, 
                            y_train, 
                            validation_data=(X_test, y_test), 
                            epochs=200, 
                            batch_size=100,
                            verbose=verbose)

  if verbose == 1:
    plot_accuracy('Model 3 (binary_crossentropy)', model_history)
    plot_loss('Model 3 (binary_crossentropy)', model_history)

  return model

model3 = train_model3(X_train, X_test, y_train, y_test)

# 7. Visualize the predictions with the final base model
Now it is time to start making visualisations, and see wheter the SDM is accurate.

Below we will:

1. Import the European environment data to make predicitions on.
1. Change the land climate name into categorical data, to make it easier to distinguish for machines (from words to numbers).
1. Remove allw water bodies, because butterflies can't live on water
1. Normalize the data

In [ ]:
# Prepare full Europe environment dataset
df_env = pd.read_csv(repo_path / 'data/prepared/data_env_fixed_prepared.csv', index_col=0)

# Categorize Land Climate data
df_env['land_climate'], categories_land = create_cat_values(df_env['land_climate'], 'land_climate')

# Remove all water bodies from dataset
df_env_no_water = df_env[df_env['land_climate'] != 'Water Bodies']
df_env_no_water.dropna(inplace=True)

# Normalize the data
column_to_norm = ['elevation', 'WorldClim_prec_m06', 'WorldClim_srad_m06',
       'WorldClim_tavg_m06', 'WorldClim_tmax_m06', 'WorldClim_tmin_m06',
       'WorldClim_vapr_m06', 'WorldClim_wind_m06', 'BioClim_01', 'BioClim_02',
       'BioClim_03', 'BioClim_04', 'BioClim_05', 'BioClim_06', 'BioClim_07',
       'BioClim_08', 'BioClim_09', 'BioClim_10', 'BioClim_11', 'BioClim_12',
       'BioClim_13', 'BioClim_14', 'BioClim_15', 'BioClim_16', 'BioClim_17',
       'BioClim_18', 'BioClim_19']
df_env_no_water[column_to_norm] = df_env_no_water[column_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

df_env_no_water.head()

Now, we will make predictions on how much chance a butterfly has on existing on each coordinate of the European environment dataset.

In [ ]:
def predict(model, df_env):
    # Predict likelyhood of butterfly being present on all locations in Europe
    prob = model.predict(df_env)

    judge_species0 = prob[:, 0]
    judge_species1 = prob[:, 1] 
    judge_species2 = prob[:, 2]
    judge_species3 = prob[:, 3]
    judge_species4 = prob[:, 4]

    return judge_species0, judge_species1, judge_species2, judge_species3, judge_species4

# Predict likelyhood with best model (model3)
judge_species0, judge_species1, judge_species2, judge_species3, judge_species4 = predict(model3, df_env_no_water)

These two functions are able to visualise the predictions per species

In [ ]:
# Visualisations
def plot_positives(environment, judge, boundary):
    positives = environment[judge >= boundary]

    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    ax = world.plot(color='white', edgecolor='black', figsize=(40, 40))

    gdf = gpd.GeoDataFrame(
        positives, 
        geometry=gpd.points_from_xy(positives['lon_latlon'], 
                                    positives['lat_latlon']))
    gdf.plot(ax=ax, markersize=15)

    #Define map boundaries
    plt.xlim(-25, 43)
    plt.ylim(33, 72)

    plt.show()

def plot_probability(environment, judge, boundary):
    positives = environment[judge >= boundary]
    judge_positives = judge[judge >= boundary]

    preds = positives[['lat_latlon', 'lon_latlon']]
    preds['judge'] = judge_positives

    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    ax = world.plot(color='white', edgecolor='black', figsize=(40, 40))

    gdf = gpd.GeoDataFrame(
        positives, 
        geometry=gpd.points_from_xy(preds['lon_latlon'], 
                                    preds['lat_latlon']))
    gdf.plot(ax=ax, 
             column=preds['judge'], 
             legend=True, 
             markersize=15, 
             legend_kwds={'shrink': 0.5})

    #Define map boundaries
    plt.xlim(-25, 43)
    plt.ylim(33, 72)

    plt.show()

    # Returns DataFrame with locations and prediction certainty
    return preds

Below you can see the visualisations of the five butterflies, predicted using our best model (the third one using binary-crossentropy).

## Species 0 (Agriades optilete)

In [ ]:
# plot_positives(df_env_no_water, judge_species0, .5)
preds = plot_probability(df_env_no_water, judge_species0, .01)

## Species 1 (Boloria titania)

In [ ]:
# plot_positives(df_env_no_water, judge_species1, .5)
preds = plot_probability(df_env_no_water, judge_species1, .01)

## Species 2 (Melanargia galathea)

In [ ]:
# plot_positives(df_env_no_water, judge_species2, .5)
preds = plot_probability(df_env_no_water, judge_species2, .01)

## Species 3 (Parnassius apollo)

In [ ]:
# plot_positives(df_env_no_water, judge_species3, .5)
preds = plot_probability(df_env_no_water, judge_species3, .01)

## Species 4 (Zerynthia rumina)

In [ ]:
# plot_positives(df_env_no_water, judge_species4, .5)
preds = plot_probability(df_env_no_water, judge_species4, .01)

## Analysis

In [ ]:
analyse(X_train, X_test, y_train, y_test, model3, 0.5)

# 8. Visualisations with SDM optimalisations

This chapter will contain information on all optimalisations that were executed. None of the optimalisations have been added to the final base model, because as ICT students it was hard to decide whether a optimalisation has a positive effect

Below you can see the evaluation on optimalisations of the SDM. The evaluation contains the following:

- SDM output
- Scatterplot
- Boxplot
- Precision, recall, and F1-score

If you want indept information about the evaluation, you are referred to the final report.

The following were the optimalisations tested:

1. Oversampling
1. Feature dropping
1. Plotting predictions within X distance of nearest observation

## 8.1. Oversampling

The visualisations above all look great, but now, let us check wheter using ovesampling will increase the model's perfomance.

So we will train our final base model (model 3) with oversampling enabled, and predict how much chance a butterfly has on existing on each coordinate of the European environment dataset again.

Oversampling is a technique that multiplace the categories with less data values and increases its data size. For this project the approach taken with oversampling was multiplying all the values to the same amount as species with most observations.

In [ ]:
model3_oversample = train_model3(X_train, X_test, y_train, y_test, oversample=True, verbose=0)
judge_species0_oversample, judge_species1_oversample, judge_species2_oversample, judge_species3_oversample, judge_species4_oversample = predict(model3_oversample, df_env_no_water)

### Normal visualisation (final base model)

In [ ]:
preds = plot_probability(df_env_no_water, judge_species0, .5)

### Visualisation with oversampling

In [ ]:
preds = plot_probability(df_env_no_water, judge_species0_oversample, .5)

As you can see oversampling improves the model's certainty percentage at places it wasn't present. This changes the output of the model.

### Analysis

In [ ]:
analyse(X_train, X_test, y_train, y_test, model3_oversample, 0.5)

## 8.2. Feature dropping (coordinates)
Now let us experiment with dropping the coordinates, to see how the output of the model changes.

In [ ]:
# Removing the coordinates from the butterflies and training the third model again
X_without_coordinates = X.drop(['obs_latitude', 'obs_longitude'], axis=1)
X_noco_train, X_noco_test, y_noco_train, y_noco_test = train_test_split(X_without_coordinates, y, test_size=0.33, random_state=42)
model3_without_coordinates = train_model3(X_noco_train, X_noco_test, y_noco_train, y_noco_test, verbose=0)

# Removing the coordinates from the environment dataset
df_env_no_water_coordinates = df_env_no_water.drop(['lat_latlon', 'lon_latlon'], axis=1)

# Making new predictions
judge_species0_without_coordinates, judge_species1_without_coordinates, judge_species2_without_coordinates, judge_species3_without_coordinates, judge_species4_without_coordinates = predict(model3_without_coordinates, df_env_no_water_coordinates)

### Normal visualisation

In [ ]:
preds = plot_probability(df_env_no_water, judge_species4, .01)

### Visualisation with feature dropping (coordinates)

In [ ]:
preds = plot_probability(df_env_no_water, judge_species4_without_coordinates, .01)

We can see that the results here have less impact on the certainty percentage, but more on the distribution of a butterfly.

### Analysis

In [ ]:
analyse(X_noco_train, X_noco_test, y_noco_train, y_noco_test, model3_without_coordinates, 0.5)

## 8.3 Plotting predictions within X distance of nearest observation

Vincent came with a request to not plot predictions that were further than distance X from the nearest observation. For the prototype we created a function which is able to draw the maximum and minimum latitude and longitude values, which then creates a rectengle with the species living habitat. It is important to know the function is not at its final stages, and will need to be researched futher, as having multiple cluster of the same species will bring problems.

In the POC the distance of 1 latitude and longitude point was used, which translates to about 110km.

In [ ]:
# Get the Max/Min Lat/Lon values for a species
def get_maxcoordinates(df, species, lat_lon_points):
    df_spec = df[df['taxon_full_name'] == species]
    maxminval = []
    latmax = df_spec['obs_latitude'].max() + lat_lon_points
    latmin = df_spec['obs_latitude'].min() - lat_lon_points
    lonmax = df_spec['obs_longitude'].max() + lat_lon_points
    lonmin = df_spec['obs_longitude'].min() - lat_lon_points
    maxminval.append(latmax)
    maxminval.append(latmin)
    maxminval.append(lonmax)
    maxminval.append(lonmin)
    return maxminval

# This is the same function as plot_probability, now filtering out the values between the maximum and minimum latitiude/longitude
def plot_probability_distancefurther_x(environment, judge, boundary, maxmin):
    positives = environment[judge >= boundary]
    judge_positives = judge[judge >= boundary]

    preds = positives[['lat_latlon', 'lon_latlon']]
    preds['judge'] = judge_positives
    preds = preds[preds["lat_latlon"] < maxmin[0]]
    preds = preds[preds["lat_latlon"] > maxmin[1]]
    preds = preds[preds["lon_latlon"] < maxmin[2]]
    preds = preds[preds["lon_latlon"] > maxmin[3]]
    preds.reset_index()

    positives = positives[positives["lat_latlon"] < maxmin[0]]
    positives = positives[positives["lat_latlon"] > maxmin[1]]
    positives = positives[positives["lon_latlon"] < maxmin[2]]
    positives = positives[positives["lon_latlon"] > maxmin[3]]
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    ax = world.plot(color='white', edgecolor='black', figsize=(40, 40))

    gdf = gpd.GeoDataFrame(
    positives,
    geometry=gpd.points_from_xy(preds['lon_latlon'],
    preds['lat_latlon']))
    gdf.plot(ax=ax,
    column=preds['judge'],
    legend=True,
    markersize=15,
    legend_kwds={'shrink': 0.5})

    #Define map boundaries
    plt.xlim(-25, 43)
    plt.ylim(33, 72)

    plt.show()

    # Returns DataFrame with locations and prediction certainty
    return preds

### Normal visualisation

In [ ]:
preds = plot_probability(df_env_no_water, judge_species4, .01)

### Visualisation with predictions within 1 distance of nearest observation

In [ ]:
maxminval = get_maxcoordinates(df, 4, 1)
preds = plot_probability_distancefurther_x(df_env_no_water, judge_species4, .5, maxminval)

This function works as expected

## All visualisations
### Visualisation base model

In [ ]:
preds = plot_probability(df_env_no_water, judge_species4, .01)

### Visualisation with oversampling

In [ ]:
preds = plot_probability(df_env_no_water, judge_species4_oversample, .5)

### Visualisation with feature dropping

In [ ]:
preds = plot_probability(df_env_no_water, judge_species4_without_coordinates, .01)

### Visualisation with predictions within 1 distance of nearest observation

In [ ]:
maxminval = get_maxcoordinates(df, 4, 1)
preds = plot_probability_distancefurther_x(df_env_no_water, judge_species4, .5, maxminval)